### Imports

In [421]:
import pandas as pd
import time
import random

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from IPython.display import display
from datetime import datetime

from dotenv import load_dotenv
import os
load_dotenv()

from utilsNew import playSound2



### Alcampo Selenium First Try =>  ❌ Fail (bot detection)

In [ ]:
# 1. Abre navegador en blanco
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 2. Abrimos web mercadona
url_alcampo ="https://www.alcampo.es/compra-online/alimentacion/c/WC10"
driver.get(url_alcampo)

#Esperamos unos segundillos
time.sleep(3)

# Aceptamos cookies
cookies = driver.find_element(By.CLASS_NAME,"cc_b_ok.auxOKButton.cookie-button")
cookies.send_keys(Keys.ENTER)

#Esperamos unos segundillos
time.sleep(3)

# Pinchamos en cualquier lado para quitar overlay z-score
expand_page = driver.find_element(By.CLASS_NAME,"ui-widget-overlay.first-time-overlay")
expand_page.click()


### Expandir Categorias

In [ ]:
category_open = driver.find_element(By.CSS_SELECTOR,"#content > div.span-24 > div.span-4.facetNavigation > div.caja.caja-reposo.slideGridDiv2.hiddenNav > div.facet.price-filter-hidden.facetaCategoria > div.facetHead.itemCerrado > h3 > a")
category_open.click()

time.sleep(3)

expand_category = driver.find_element(By.CSS_SELECTOR,"#content > div.span-24 > div.span-4.facetNavigation > div.caja.caja-reposo.slideGridDiv2.hiddenNav > div.facet.price-filter-hidden.facetaCategoria.coultarItemFacetPriceDesktop > div.facetValues > div.topFacetValues > span > a")
expand_category.click()

In [ ]:
# Recorrer cada categoria

for i in range(1,15):

    header_a_pinchar = driver.find_elements(By.CSS_SELECTOR,\
        f"#content > div.span-24 > div.span-4.facetNavigation > div.caja.caja-reposo.slideGridDiv2.hiddenNav > div.facet.price-filter-hidden.facetaCategoria.coultarItemFacetPriceDesktop > div.facetValues > div.allFacetValues > ul > li:nth-child({i}) > a")
    header_a_pinchar[0].click()
    time.sleep(5)
    print(header_a_pinchar[0].text.split("\n"))

## Imports

In [231]:
from bs4 import BeautifulSoup as bs
from utils import alcampo
import requests
import pandas as pd
import numpy as np

## Step 1: Get Data.

### Alcampo Web Scraping Script using BeautifulSoup

#### 1. Open web url

In [556]:
url ="https://www.alcampo.es/compra-online/alimentacion/leche-huevos-yogures-y-lacteos/c/W16/headers"

proxies = {
  "http": "PROXY"
}

r = requests.get('https://www.alcampo.es/compra-online/alimentacion/leche-huevos-yogures-y-lacteos/c/W16/', proxies=proxies, verify=False)
print(r)
print(r.text)

# response = requests.get(url,params)
# response

<Response [403]>
<!DOCTYPE html>
<html lang="en-US">
<head>
    <title>Just a moment...</title>
    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=Edge">
    <meta name="robots" content="noindex,nofollow">
    <meta name="viewport" content="width=device-width,initial-scale=1">
    <link href="/cdn-cgi/styles/challenges.css" rel="stylesheet">
    

</head>
<body class="no-js">
    <div class="main-wrapper" role="main">
    <div class="main-content">
        <h1 class="zone-name-title h1">
            <img class="heading-favicon" src="/favicon.ico" alt="Icon for www.alcampo.es"
                 onerror="this.onerror=null;this.parentNode.removeChild(this)">
            www.alcampo.es
        </h1>
        <h2 class="h2" id="challenge-running">
            Checking if the site connection is secure
        </h2>
        <noscript>
            <div id="challenge-error-title">
                <div class="h2">
            

/Users/ignaciogarciabarrero/opt/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn((


In [229]:
url ="https://www.alcampo.es/compra-online/alimentacion/c/WC10?q=%3Arelevance%3AcategoryChild%3AW16"



API_KEY = os.environ["SCARPE_OPS_API_KEY"]

response = requests.get(
  url='https://proxy.scrapeops.io/v1/',
  params={
      'api_key': API_KEY,
      'url': url, 
  },
)

print(response)

html = response.content
soup = bs(html,"lxml")
soup

<Response [200]>


<!DOCTYPE html>
<html lang="es">
<head><link as="font" crossorigin="" href="/compra-online/_ui/desktop/theme-alcampo/css/fonts/fontawesome-webfont.woff2?v=4.3.0" rel="preload" type="font/woff2"/>
<title>
		Alimentación | Alcampo Compra Online </title>
<script data-cfasync="false" serverstartup="on" src="/compra-online/_ui/shared/js/cookie-consent-3.1.0.js" type="text/javascript"></script>
<script id="scriptCookies" type="c542b8db35002d8ebc051ad3-text/javascript">
        var labelCookies = {title: "Nosotros también tenemos cookies…",
            masinfo: "Más info",
            privacidad: "política de privacidad",
            politica: "política de cookies",
            homep1: "El uso de cookies en Alcampo sirve para proporcionarte la mejor experiencia de compra posible en nuestra página web. Las cookies son básicas para el funcionamiento óptimo de la página. También las usamos para realizar mejoras en la web o proponer contenidos personalizados. En definitiva tratamos de ofrecerte u

In [230]:
products = soup.find_all("div",class_="productGridItemContainer")
prices = soup.find_all("div",class_="priceContainer")

dict_alcampo = {"Name":[],"Price":[], "Image":[]}

for product in products:

    dict_alcampo["Name"].append(product.find("span").get_text(strip=True))
    dict_alcampo["Image"].append(product.find("img")["src"])

for price in prices:
    dict_alcampo["Price"].append(price.find_all("span")[2].get_text(strip=True))

print(len(dict_alcampo["Name"]))
print(len(dict_alcampo["Image"]))
print(len(dict_alcampo["Price"]))


df_alcampo = pd.DataFrame(dict_alcampo)
df_alcampo

48
48
48


,Name,Price,Image
0,Bebida vegetal de avena SANTIVERI CALCIAVENA 1 l.,"2,61 €(2,61 €/Litro)",https://www.alcampo.es/media/h34/h79/115375081...
1,Leche semidesnatada de vaca PRODUCTO ALCAMPO 6...,"5,40 €(0,90 €/Litro)",https://www.alcampo.es/media/h13/h14/118842518...
2,Leche entera de vaca PRODUCTO ALCAMPO 6 x 1 l.,"5,40 €(0,90 €/Litro)",https://www.alcampo.es/media/h0a/he2/118842514...
3,Leche desnatada de vaca PRODUCTO ALCAMPO 6x 1 l.,"5,34 €(0,89 €/Litro)",https://www.alcampo.es/media/h51/h3f/118842522...
4,Leche entera L.R. 6 x 1 l.,"5,28 €(0,88 €/Litro)",https://www.alcampo.es/media/h37/h6a/104576930...
5,Leche semidesnatada L.R. 6 x 1 l.,"5,28 €(0,88 €/Litro)",https://www.alcampo.es/media/he0/h06/104577404...
6,Leche desnatada L.R. 6 x 1 l.,"5,28 €(0,88 €/Litro)",https://www.alcampo.es/media/h6f/h03/104577576...
7,"Nata líquida ligera para cocinar, PRODUCTO ALC...","0,69 €(3,45 €/Litro)",https://www.alcampo.es/media/he5/hc5/102407521...
8,Leche semidesnatada de vaca PRODUCTO ALCAMPO 1 l.,"0,90 €(0,90 €/Litro)",https://www.alcampo.es/media/hdd/hec/956547855...
9,Leche de vaca semidesnatada L.R. 1 l.,"0,88 €(0,88 €/Litro)",https://www.alcampo.es/media/h7d/h27/115935077...


In [210]:
prices = soup.find_all("div",class_="priceContainer")
# print(prices)



for price in prices:
    print(price.find_all("span")[2].get_text(strip=True))

2,61 €(2,61 €/Litro)
5,40 €(0,90 €/Litro)
5,40 €(0,90 €/Litro)
5,34 €(0,89 €/Litro)
5,28 €(0,88 €/Litro)
5,28 €(0,88 €/Litro)
5,28 €(0,88 €/Litro)
0,69 €(3,45 €/Litro)
0,90 €(0,90 €/Litro)
0,88 €(0,88 €/Litro)
6,66 €(1,11 €/Litro)
1,06 €(2,12 €/Kg)
0,90 €(0,90 €/Litro)
(2,45 €/Docena)
(2,58 €/Kg)
3,53 €(3,53 €/Docena)
0,88 €(0,88 €/Litro)
0,89 €(0,89 €/Litro)
0,99 €(0,99 €/Litro)
1,11 €(1,11 €/Litro)
6,66 €(1,11 €/Litro)
5,94 €(0,99 €/Litro)
1,69 €(1,69 €/Kg)
0,99 €(0,99 €/Litro)
0,99 €(0,99 €/Litro)
4,75 €(1,90 €/Docena)
1,52 €(1,82 €/Docena)
1,44 €(2,88 €/Litro)
1,11 €(1,11 €/Litro)
1,49 €(1,24 €/Litro)
5,94 €(0,99 €/Litro)
7,20 €(1,20 €/Litro)
1,09 €(2,18 €/Kg)
1€(1,00 €/Litro)
2,25 €(2,25 €/Docena)
7,20 €(1,20 €/Litro)
1,55 €(1,55 €/Kg)
6€(1,00 €/Litro)
1,04 €(2,08 €/Kg)
0,70 €(1,40 €/Kg)
1,69 €(1,69 €/Kg)
7,80 €(1,30 €/Litro)
1€(1,00 €/Litro)
1,71 €(1,14 €/Litro)
0,88 €(0,88 €/Litro)
2,33 €(4,66 €/Docena)
0,37 €(2,96 €/Kg)
3,95 €(3,95 €/Docena)


### Pedir datos de varias pestañas dentro de la misma url (distinto & page)

In [496]:
url ="https://www.alcampo.es/compra-online/frescos/sushi/c/W140502?q=%3Arelevance&page=0"

response = requests.get(url='https://proxy.scrapeops.io/v1/',params={'api_key': os.environ["SCARPE_OPS_API_KEY"],'url': url, },)

print(response)

html = response.content
soup = bs(html,"lxml")
soup

<Response [200]>


<!DOCTYPE html>
<html lang="es">
<head><link as="font" crossorigin="" href="/compra-online/_ui/desktop/theme-alcampo/css/fonts/fontawesome-webfont.woff2?v=4.3.0" rel="preload" type="font/woff2"/>
<title>
		Sushi | Alcampo Compra Online </title>
<script data-cfasync="false" serverstartup="on" src="/compra-online/_ui/shared/js/cookie-consent-3.1.0.js" type="text/javascript"></script>
<script id="scriptCookies" type="aa0e2523c47c4fe1f4a51af5-text/javascript">
        var labelCookies = {title: "Nosotros también tenemos cookies…",
            masinfo: "Más info",
            privacidad: "política de privacidad",
            politica: "política de cookies",
            homep1: "El uso de cookies en Alcampo sirve para proporcionarte la mejor experiencia de compra posible en nuestra página web. Las cookies son básicas para el funcionamiento óptimo de la página. También las usamos para realizar mejoras en la web o proponer contenidos personalizados. En definitiva tratamos de ofrecerte un exper

In [383]:
product_not_found = soup.find("div",class_="totalResults").get_text(strip=True, separator='').split(" ")[0]
print(product_not_found)
# while product_not_found != "0":


1.584


In [527]:
alimentacion = {
    "Leche Huevos Yogures y Lácteos": "https://www.alcampo.es/compra-online/alimentacion/leche-huevos-yogures-y-lacteos/c/W16?q=%3Arelevance&page=",
    "Aceite Vinagre Salsas Especias" : "https://www.alcampo.es/compra-online/alimentacion/aceite-vinagre-salsas-especias/c/W18?q=%3Arelevance&page=",
    "Desayuno y Merienda": "https://www.alcampo.es/compra-online/alimentacion/desayuno-y-merienda/c/W10?q=%3Arelevance&page=",
    "Conservas": "https://www.alcampo.es/compra-online/alimentacion/conservas/c/W1004?q=%3Arelevance&page=",
    "Comida Internacional":"https://www.alcampo.es/compra-online/alimentacion/comida-internacional/c/W9410?q=%3Arelevance&page=",
    "Aperitivos, aceitunas y frutos secos": "https://www.alcampo.es/compra-online/alimentacion/aperitivos-aceitunas-y-frutos-secos/c/W120?q=%3Arelevance&page=",
    "Nutrición deportiva":"https://www.alcampo.es/compra-online/alimentacion/nutricion-deportiva/c/W12?q=%3Arelevance&page=",
    "Caldos, Pasta, Arroz Legumbres Puré":"https://www.alcampo.es/compra-online/alimentacion/caldos-pasta-arroz-legumbres-pure/c/W140?q=%3Arelevance&page=",
    "Azúcar Harina Masas y Pan": "https://www.alcampo.es/compra-online/alimentacion/azucar-harina-masas-y-pan/c/W1009?q=%3Arelevance&page=",
    "Alimentos Especiales": "https://www.alcampo.es/compra-online/alimentacion/alimentos-especiales-/c/W33?q=%3Arelevance&page=",
    "Veganos": "https://www.alcampo.es/compra-online/alimentacion/veganos/c/W09112021?q=%3Arelevance&page=",
    "Supermercado Ecológico": "https://www.alcampo.es/compra-online/alimentacion/supermercado-ecologico/c/W26112021?q=%3Arelevance&page="
}

contador = 5

for i in alimentacion.items():
    
    url = i[1]
    print(i[1])






https://www.alcampo.es/compra-online/alimentacion/nutricion-deportiva/c/W12?q=%3Arelevance&page=5


### Alimentación

In [544]:
alimentacion = {
    # "Leche Huevos Yogures y Lácteos": "https://www.alcampo.es/compra-online/alimentacion/leche-huevos-yogures-y-lacteos/c/W16?q=%3Arelevance&page=",
    # "Aceite Vinagre Salsas Especias" : "https://www.alcampo.es/compra-online/alimentacion/aceite-vinagre-salsas-especias/c/W18?q=%3Arelevance&page=",
    # "Desayuno y Merienda": "https://www.alcampo.es/compra-online/alimentacion/desayuno-y-merienda/c/W10?q=%3Arelevance&page=",
    # "Conservas": "https://www.alcampo.es/compra-online/alimentacion/conservas/c/W1004?q=%3Arelevance&page=",
    # "Comida Internacional":"https://www.alcampo.es/compra-online/alimentacion/comida-internacional/c/W9410?q=%3Arelevance&page=",
    # "Aperitivos, aceitunas y frutos secos": "https://www.alcampo.es/compra-online/alimentacion/aperitivos-aceitunas-y-frutos-secos/c/W120?q=%3Arelevance&page=",
    # "Nutrición deportiva":"https://www.alcampo.es/compra-online/alimentacion/nutricion-deportiva/c/W12?q=%3Arelevance&page=",
    # "Caldos, Pasta, Arroz Legumbres Puré":"https://www.alcampo.es/compra-online/alimentacion/caldos-pasta-arroz-legumbres-pure/c/W140?q=%3Arelevance&page=",
    # "Azúcar Harina Masas y Pan": "https://www.alcampo.es/compra-online/alimentacion/azucar-harina-masas-y-pan/c/W1009?q=%3Arelevance&page=",
    # "Alimentos Especiales": "https://www.alcampo.es/compra-online/alimentacion/alimentos-especiales-/c/W33?q=%3Arelevance&page=",
    "Veganos": "https://www.alcampo.es/compra-online/alimentacion/veganos/c/W09112021?q=%3Arelevance&page=",
    # "Supermercado Ecológico": "https://www.alcampo.es/compra-online/alimentacion/supermercado-ecologico/c/W26112021?q=%3Arelevance&page="
}



dict_alcampo = {"Category": [], "Name":[],"Price":[],"Stock":[], "Image":[]}

for i in alimentacion.items():

    categoria = i[0]
    url_to_scrape = i[1]
    contador = 0
    product_not_found = 10

    while product_not_found != "0":
        
        url = url_to_scrape+ f"{contador}"
        response = requests.get(url='https://proxy.scrapeops.io/v1/',params={'api_key': os.environ["SCARPE_OPS_API_KEY"],'url': url, },)
        html = response.content
        soup = bs(html,"lxml")
     
        print(categoria,url)
        try:
            product_not_found = soup.find("div",class_="totalResults").get_text(strip=True, separator='').split(" ")[0]
        except:
            print("Got an attribute error on:",url)

        if product_not_found =="0":
            print("No hay más ventanas")
            break
        
        else:
            products = soup.find_all("div",class_="productGridItemContainer")
            prices = soup.find_all("div",class_="priceContainer")
            stock_available = soup.find_all("div",class_="cart add")

            for product in products:

                dict_alcampo["Name"].append(product.find("span").get_text(strip=True))
                dict_alcampo["Image"].append(product.find("img")["src"])

                dict_alcampo["Category"].append(categoria)

            for price in prices:
                try:
                    dict_alcampo["Price"].append(price.find_all("span")[2].get_text(strip=True))
                except:
                    print("List index out of range")
                    dict_alcampo["Price"].append("NaN")

            for stock in stock_available:
                
                stock_yes_no = stock.find_all("button")[2].get_text(strip=True).split(" "[0])
                
                if stock_yes_no[0]=="Sin":
                    dict_alcampo["Stock"].append("No")
                else:
                    dict_alcampo["Stock"].append("Yes")
                              
            print( "Category:", len(dict_alcampo["Category"]), "Name:", len(dict_alcampo["Name"]) ,"Image:",len(dict_alcampo["Image"]),"Price:", len(dict_alcampo["Price"]), "Stock",len(dict_alcampo["Stock"])    )
            
        contador+=1

df_alcampo = pd.DataFrame(dict_alcampo)
playSound2()
df_alcampo


Veganos https://www.alcampo.es/compra-online/alimentacion/veganos/c/W09112021?q=%3Arelevance&page=0
Category: 48 Name: 48 Image: 48 Price: 48 Stock 48
Veganos https://www.alcampo.es/compra-online/alimentacion/veganos/c/W09112021?q=%3Arelevance&page=1
Category: 96 Name: 96 Image: 96 Price: 96 Stock 96
Veganos https://www.alcampo.es/compra-online/alimentacion/veganos/c/W09112021?q=%3Arelevance&page=2
Category: 144 Name: 144 Image: 144 Price: 144 Stock 144
Veganos https://www.alcampo.es/compra-online/alimentacion/veganos/c/W09112021?q=%3Arelevance&page=3
List index out of range
Category: 192 Name: 192 Image: 192 Price: 192 Stock 192
Veganos https://www.alcampo.es/compra-online/alimentacion/veganos/c/W09112021?q=%3Arelevance&page=4
Category: 240 Name: 240 Image: 240 Price: 240 Stock 240
Veganos https://www.alcampo.es/compra-online/alimentacion/veganos/c/W09112021?q=%3Arelevance&page=5
List index out of range
Category: 288 Name: 288 Image: 288 Price: 288 Stock 288
Veganos https://www.alcamp

,Category,Name,Price,Stock,Image
0,Veganos,Bebida de avena con alto contenido de calcio y...,"5,94 €(0,99 €/Litro)",Yes,https://www.alcampo.es/media/hbc/hc7/114853804...
1,Veganos,"Bebida de soja sin gluten, enriquecida con cal...","5,34 €(0,89 €/Litro)",Yes,https://www.alcampo.es/media/h2b/hc3/107936543...
2,Veganos,Bebida de avena con alto contenido de calcio y...,"0,99 €(0,99 €/Litro)",Yes,https://www.alcampo.es/media/hc3/h79/109357267...
3,Veganos,"Bebida de almendras 100% vegetal, sin azúcar n...","2,10 €(2,10 €/Litro)",Yes,https://www.alcampo.es/media/h31/h28/114171706...
4,Veganos,Bebida de avena ALCAMPO ECOLÓGICO brick de 1 l...,"11,22 €(0,18 €/100ml)",Yes,https://www.alcampo.es/media/hb5/h66/111936354...
...,...,...,...,...,...
394,Veganos,"Vino tinto frinzzante 0,0% alcohol WIN 0.0 bot...",8€,Yes,https://www.alcampo.es/media/h6b/hbb/933832897...
395,Veganos,Tofu ecológico ECOCESTA 440 g.,"4,32 €(9,81 €/Kg)",Yes,https://www.alcampo.es/media/h60/hf8/111513052...
396,Veganos,Vino tinto crianza con denominación de origen ...,"8,44 €",Yes,https://www.alcampo.es/media/hd7/h2c/112060643...
397,Veganos,"Puré de patata receta casera, 100% natural, si...","2,56 €(12,48 €/Kg)",Yes,https://www.alcampo.es/media/hb7/h07/997146565...


#### CREATE CSV + SAVE AS EXCEL


In [546]:
df_alcampo['Shop'] = 'Alcampo'

today = pd.Timestamp.now()
df_alcampo['Date'] = today
df_alcampo



,Category,Name,Price,Stock,Image,Shop,Date
0,Veganos,Bebida de avena con alto contenido de calcio y...,"5,94 €(0,99 €/Litro)",Yes,https://www.alcampo.es/media/hbc/hc7/114853804...,Alcampo,2023-03-08 19:51:25.159831
1,Veganos,"Bebida de soja sin gluten, enriquecida con cal...","5,34 €(0,89 €/Litro)",Yes,https://www.alcampo.es/media/h2b/hc3/107936543...,Alcampo,2023-03-08 19:51:25.159831
2,Veganos,Bebida de avena con alto contenido de calcio y...,"0,99 €(0,99 €/Litro)",Yes,https://www.alcampo.es/media/hc3/h79/109357267...,Alcampo,2023-03-08 19:51:25.159831
3,Veganos,"Bebida de almendras 100% vegetal, sin azúcar n...","2,10 €(2,10 €/Litro)",Yes,https://www.alcampo.es/media/h31/h28/114171706...,Alcampo,2023-03-08 19:51:25.159831
4,Veganos,Bebida de avena ALCAMPO ECOLÓGICO brick de 1 l...,"11,22 €(0,18 €/100ml)",Yes,https://www.alcampo.es/media/hb5/h66/111936354...,Alcampo,2023-03-08 19:51:25.159831
...,...,...,...,...,...,...,...
394,Veganos,"Vino tinto frinzzante 0,0% alcohol WIN 0.0 bot...",8€,Yes,https://www.alcampo.es/media/h6b/hbb/933832897...,Alcampo,2023-03-08 19:51:25.159831
395,Veganos,Tofu ecológico ECOCESTA 440 g.,"4,32 €(9,81 €/Kg)",Yes,https://www.alcampo.es/media/h60/hf8/111513052...,Alcampo,2023-03-08 19:51:25.159831
396,Veganos,Vino tinto crianza con denominación de origen ...,"8,44 €",Yes,https://www.alcampo.es/media/hd7/h2c/112060643...,Alcampo,2023-03-08 19:51:25.159831
397,Veganos,"Puré de patata receta casera, 100% natural, si...","2,56 €(12,48 €/Kg)",Yes,https://www.alcampo.es/media/hb7/h07/997146565...,Alcampo,2023-03-08 19:51:25.159831


In [547]:
df_alcampo.to_csv("./data/alcampo.csv",index=False)

read_file = pd.read_csv("./data/alcampo.csv")
read_file.to_excel("./data/alcampo.xlsx", index = None, header=True)

In [367]:
products = soup.find_all("div",class_="productGridItemContainer")
products_patrocinados = soup.find_all("div",class_="productGridItemContainer.rm-product.inGridL4.SP")



for product in products[13:16]:

    dict_alcampo["Name"].append(product.find("span").get_text(strip=True))
    dict_alcampo["Image"].append(product.find("img")["src"])
    # print(product.find("span").get_text(strip=True))


prices = soup.find_all("div",class_="precioFila")

preciossss = soup.find_all("div",class_="priceContainer")
# print(preciossss)
#content > div.span-24 > div.span-20.last.slideGridDiv3 > div:nth-child(6) > div > div.productGridItemContainer.rm-product.inGridL4.SP
#content > div.span-24 > div.span-20.last.slideGridDiv3 > div:nth-child(6) > div > div.productGridItemContainer.rm-product.inGridL6.SP

contador = 1
for price in prices[13:16]:

    print(contador,": ",price.find_all("span")[2].get_text(strip=True))
        
    # dict_alcampo["Price"].append(price.find_all("span")[1].get_text(strip=True))
    # print(contador,": ",price.find_all("span")[1].get_text(strip=True))
    contador+=1
        

    # print(len(dict_alcampo["Name"]))
    # print(len(dict_alcampo["Image"]))
    # print(len(dict_alcampo["Price"]))

Huevos frescos de categoria A y clase L PRODUCTO ALCAMPO 12 uds.
Yogur estilo griego con stracciatella PRODUCTO ALCAMPO 4 x 125 g.
Huevos camperos frescos de categoria A y clase M - L PAZO DE VILANE 12 uds.
1 :  (2,45 €/Docena)
2 :  (2,58 €/Kg)
3 :  3,53 €(3,53 €/Docena)


In [502]:
stock_available = soup.find_all("div",class_="cart add")

stockssss = []

for stock in stock_available:

    stock_yes_no = stock.find_all("button")[2].get_text(strip=True).split(" "[0])

    if stock_yes_no[0]=="Sin":
        stockssss.append("No")
        
    else:
        stockssss.append("yes")
        
    # print(stock.find_all("button")[2].get_text(strip=True).split(" "[0]))
    #print(stock.find_all("button")[2].get_text(strip=True).split(" ")[:2])

print(len(stockssss))

40


In [510]:
df_alcampo

,Name,Price,Stock,Image
0,Bandeja 6 sushi salmón y 8 california salmón y...,"12,25 €(35,00 €/Kg)",Yes,https://www.alcampo.es/media/h95/h61/119409061...
1,"Bandeja 6 sushi salmón, 6 maki pepino y 6 cris...","12,95 €(26,42 €/Kg)",Yes,https://www.alcampo.es/media/ha4/h40/119409089...
2,"Bandeja 2 sushi salmón, 2 sushi atún y 8 calif...","9,95 €(36,85 €/Kg)",No,https://www.alcampo.es/media/h50/haf/119408698...
3,"Bandeja 6 sushi salmón, 6 maki salmón y 6 cris...","13,95 €(39,85 €/Kg)",Yes,https://www.alcampo.es/media/h3b/h40/119409106...
4,Bandeja 8 California salmón y 4 de atún con ma...,"7,50 €(37,50 €/Kg)",Yes,https://www.alcampo.es/media/h14/h2a/119409134...
5,Bandeja 4 California salmón y 3 sushi salmón S...,"6,50 €(29,54 €/Kg)",Yes,https://www.alcampo.es/media/hbe/h4c/119409120...
6,Bandeja California salmón SUSHI GOURMET 8 uds,"5,50 €(42,30 €/Kg)",Yes,https://www.alcampo.es/media/h17/h66/119409176...
7,Raviolis japoneses rellenos de pollo SUSHI GOU...,"6,95 €(49,64 €/Kg)",No,https://www.alcampo.es/media/hb2/h09/119409075...
8,Arroz tartar SUSHI GOURMET 250 gr.,"7,95 €(31,80 €/Kg)",Yes,https://www.alcampo.es/media/h2a/h27/119409206...
9,Aperitivo makis variado SUSHITA 41 pieza.,"11,90 €(19,34 €/Kg)",No,https://www.alcampo.es/media/h04/h4a/895995251...
